In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
import itertools

# Load the data
data_raw = pd.read_csv('MMM_full.csv')
data_raw['Date'] = pd.to_datetime(data_raw['Date'], format='%d/%m/%y')
data_raw.set_index('Date', inplace=True)

def calculate_mml(fisher_info_matrix, log_likelihood, k):
    # Constants
    EULER_CONSTANT = 0.577215664901532860606512090082402431042
    PI = np.pi
    LOG_2PI = np.log(2 * PI + 1e-10)
    fisher_det = np.linalg.det(fisher_info_matrix)

    lattice_constant = -0.5 * k * LOG_2PI + 0.5 * np.log(k * PI) - EULER_CONSTANT
    # log_lattice_constant = -1*LOG_2PI + (1/k)*np.log(k * PI) - ((2*EULER_CONSTANT)/k) - 1

    comp1 = 0 # log(1) = 0
    comp2 = 0.5*np.log(fisher_det)
    comp3 = lattice_constant
    comp4 = -1*log_likelihood
    mml = comp1 + comp2 + comp3 + comp4
    return mml

# List of stocks
stocks = ['MMM', 'KO', 'JPM', 'INTC', 'IBM', 'GS', 'CSCO', 'BA', 'AAPL', 'JNJ']

# Initialize lists to store results
results = []

T = 3

# Suppress warnings
warnings.filterwarnings("ignore")

for stock in stocks:
    print(f"Processing stock: {stock}")
    # Split the data
    train_ratio = 0.8
    train_len = int(len(data_raw) * train_ratio)
    train_data = data_raw[stock].iloc[:train_len]
    test_data = data_raw[stock].iloc[train_len:train_len+T]  # Forecasting T=3 steps ahead

    # Determine d using ADF test
    X = train_data.values
    result = adfuller(X)
    d = 0
    if result[1] > 0.05:
        for i in range(1, 3):
            X_diff = np.diff(X, n=i)
            result = adfuller(X_diff)
            if result[1] <= 0.05:
                d = i
                break
    else:
        d = 0

    # Grid search over p and q
    p_range = range(0, 5)
    q_range = range(0, 5)
    pdq_combinations = list(itertools.product(p_range, [d], q_range))

    # Initialize variables to store the best model parameters
    best_aic = np.inf
    best_bic = np.inf
    best_hqic = np.inf
    best_mml = np.inf
    best_aic_order = None
    best_bic_order = None
    best_hqic_order = None
    best_mml_order = None

    # Store AIC, BIC, HQIC values for averaging later
    aic_values = []
    bic_values = []
    hqic_values = []
    mml_values = []

    for order in pdq_combinations:
        model = ARIMA(train_data, order=order)
        model_fit = model.fit()
        aic = model_fit.aic
        bic = model_fit.bic
        hqic = model_fit.hqic

        aic_values.append(aic)
        bic_values.append(bic)
        hqic_values.append(hqic)

        k = len(model_fit.params)
        # cov_matrix = model_fit.cov_params()
        hessian = model.hessian(model_fit.params)
        fisher_info_matrix = -np.linalg.inv(hessian)
        log_likelihood = model_fit.llf
        # print(fisher_info_matrix)
        # fisher_info_matrix = np.linalg.inv(cov_matrix)
        mml = calculate_mml(fisher_info_matrix, log_likelihood, k)

        # print(best_aic)
        if aic < best_aic:
            best_aic = aic
            best_aic_order = order

        if bic < best_bic:
            best_bic = bic
            best_bic_order = order

        if hqic < best_hqic:
            best_hqic = hqic
            best_hqic_order = order

        if mml < best_mml:
            best_mml = mml
            best_mml_order = order

    # Forecasting with the best models
    # Best AIC model
    model_aic = ARIMA(train_data, order=best_aic_order)
    model_aic_fit = model_aic.fit()
    forecast_aic = model_aic_fit.forecast(steps=T)
    rmse_aic = np.sqrt(mean_squared_error(test_data, forecast_aic))

    # Best BIC model
    model_bic = ARIMA(train_data, order=best_bic_order)
    model_bic_fit = model_bic.fit()
    forecast_bic = model_bic_fit.forecast(steps=T)
    rmse_bic = np.sqrt(mean_squared_error(test_data, forecast_bic))

    # Best HQIC model
    model_hqic = ARIMA(train_data, order=best_hqic_order)
    model_hqic_fit = model_hqic.fit()
    forecast_hqic = model_hqic_fit.forecast(steps=T)
    rmse_hqic = np.sqrt(mean_squared_error(test_data, forecast_hqic))

    # Best MML model
    model_mml = ARIMA(train_data, order=best_mml_order)
    model_mml_fit = model_mml.fit()
    forecast_mml = model_mml_fit.forecast(steps=T)
    rmse_mml = np.sqrt(mean_squared_error(test_data, forecast_mml))

    # Append results
    results.append({
        'Stock': stock,
        'Best AIC': best_aic,
        'Best AIC Order': best_aic_order,
        'RMSE AIC': rmse_aic,
        'Best BIC': best_bic,
        'Best BIC Order': best_bic_order,
        'RMSE BIC': rmse_bic,
        'Best HQIC': best_hqic,
        'Best HQIC Order': best_hqic_order,
        'RMSE HQIC': rmse_hqic,
        'Best MML': best_mml,
        'Best MML Order': best_mml_order,
        'RMSE MML': rmse_mml,
        'AIC Values': aic_values,
        'BIC Values': bic_values,
        'HQIC Values': hqic_values,
        'MML Values': mml_values
    })

Processing stock: MMM
Processing stock: KO


In [2]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate total averages
average_aic = results_df['RMSE AIC'].mean()
average_bic = results_df['RMSE BIC'].mean()
average_hqic = results_df['RMSE HQIC'].mean()
average_mml87 = results_df['RMSE MML'].mean()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'AIC': [round(average_aic, T)],
    'BIC': [round(average_bic, T)],
    'HQ': [round(average_hqic, T)],
    'MML': [round(average_mml87, T)],
}, index=['T='+str(T)])

print("\nSummary of Criterion Averages:")
print(summary_df)


Summary of Criterion Averages:
             AIC        BIC         HQ        MML
T=200  30.236102  30.674109  30.324292  29.278946
